<a href="https://colab.research.google.com/github/MateusBrasileiroNato/data_science/blob/main/School_Example_Data_Analysis_(Pandas).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import html5lib
import seaborn as sns
!pip3 install html5lib
!pip3 install lxml
!pip3 install sqlalchemy

nomes_f = pd.read_json('https://servicodados.ibge.gov.br/api/v1/censos/nomes/ranking?qtd=20&sexo=f')
nomes_m = pd.read_json('https://servicodados.ibge.gov.br/api/v1/censos/nomes/ranking?qtd=20&sexo=m')

In [ ]:
nomes_f

In [ ]:
nomes_m

In [ ]:
print('Quantidade de nomes: ' + str(len(nomes_f) + len(nomes_m)))

In [15]:
frames = [nomes_f, nomes_m]

In [34]:
nomes = pd.concat(frames)['nome'].to_frame()

In [ ]:
nomes.sample(5)

In [71]:
# CREATING EMAILS FOR USERS

np.random.seed(15)

In [ ]:
total_alunos = len(nomes)

In [115]:
nomes['id_aluno'] = np.random.permutation(total_alunos) + 1

In [54]:
dominios = ['@dominiodoemmail.com.br', '@servicodoemail.com']
nomes['dominio'] = np.random.choice(dominios, total_alunos)
nomes['email'] = nomes.nome.str.cat(nomes.dominio).str.lower()

In [ ]:
# CREATING COURSES

url = 'http://tabela-cursos.herokuapp.com/index.html'
cursos = pd.read_html(url)
cursos = cursos[0]
cursos

In [ ]:
# RENAMING AND ALTERING COURSES INDEX

cursos = cursos.rename(columns = {'Nome do curso': 'nome_do_curso'})
cursos['id'] = cursos.index + 1
cursos = cursos.set_index('id')
cursos

In [ ]:
# ANALYSING ENROLLMENTS

nomes['matriculas'] = np.ceil(np.random.exponential(size = total_alunos) * 1.5).astype(int)
nomes.matriculas.describe()

In [ ]:
nomes.matriculas.value_counts()

In [ ]:
sns.distplot(nomes.matriculas)

In [112]:
# SELECTING COURSES

todas_matriculas = []
x = np.random.rand(20)
prob = x / sum(x)

In [ ]:
for index, row in nomes.iterrows():

    id = row.id_aluno
    matriculas = row.matriculas

    for i in range(matriculas):
        
        mat = [id, np.random.choice(cursos.index, p = prob)]
        todas_matriculas.append(mat)

matriculas = pd.DataFrame(todas_matriculas, columns = ['id_aluno', 'id_curso'])
matriculas.head(5)

In [148]:
matriculas_por_curso = matriculas.groupby('id_curso').count().join(cursos['nome_do_curso']).rename(columns = {'id_aluno': 'quantidade_de_alunos'})

In [ ]:
matriculas_por_curso

In [ ]:
# TO CSV

matriculas_por_curso.to_csv('matriculas_por_curso.csv', index = False)
pd.read_csv('matriculas_por_curso.csv')

In [ ]:
# TO JSON

matriculas_json = matriculas_por_curso.to_json()
matriculas_json

In [ ]:
# TO HTML

matriculas_html = matriculas_por_curso.to_html()
print(matriculas_html)

In [ ]:
# TO SQL

from sqlalchemy import create_engine, MetaData, Table

engine = create_engine('sqlite:///:memory:') 
matriculas_por_curso.to_sql('matriculas', engine)
print(engine.table_names())

In [ ]:
# SQL PANDAS QUERY TYPES EXAMPLES

query =  'select * from matriculas where quantidade_de_alunos < 20'
pd.read_sql(query, engine)

In [ ]:
todas_matriculas = pd.read_sql_table('matriculas', engine, columns = ['nome_do_curso', 'quantidade_de_alunos'])
todas_matriculas.query('quantidade_de_alunos < 10')

In [ ]:
# QUERY RESULT TO SQL

todas_matriculas.to_sql('todas_matriculas', con = engine)
print(engine.table_names())

In [222]:
# NEXT CREW EXAMPLE

id_curso = 16
proxima_turma = matriculas.query("id_curso == {}".format(id_curso))

In [ ]:
proxima_turma = proxima_turma.set_index('id_aluno').join(nomes.set_index('id_aluno'))['nome'].to_frame()
proxima_turma

In [ ]:
nome_curso = cursos.loc[id_curso]
nome_curso

In [ ]:
nome_curso = nome_curso.nome_do_curso
nome_curso

In [ ]:
proxima_turma.rename(columns = {'nome':'Alunos do curso de {}'.format(nome_curso)})

In [ ]:
# TO EXCEL

proxima_turma.to_excel('proxima_turma.xlsx', index = False)
pd.read_excel('proxima_turma.xlsx')